# Device patch base
A Kubeflow pipeline running Ansible playbook from a git repo on a subset of devices

This Notebook is meant to run in a Notebook Server on the Teknoir platform

## Define components

In [87]:
import kfp
import kfp.dsl as dsl
import kfp.components as comp
import uuid
import os
import json
from kubernetes.client.models import V1EnvVar, V1Volume, V1EmptyDirVolumeSource


prepare = comp.load_component_from_text("""
name: Prepare
description: Preparations for a playbook

inputs:
- {name: namespace, type: String}
- {name: project_id, type: String}
- {name: ansible_limit, type: String}
- {name: playbook_git_repo, type: String}

outputs:
- {name: kube_config, type: Data}
- {name: playbook_dir, type: Data}
- {name: static_inventory, type: Data}
- {name: access_token, type: Data}
- {name: devices, type: Data}
- {name: tunnels_created, type: Data}

implementation:
  container:
    image: gcr.io/teknoir/edgebuild-kfp
    args:
    - {inputValue: namespace}
    - {inputValue: project_id}
    - {outputPath: kube_config}
    - {inputValue: ansible_limit}
    - {inputValue: playbook_git_repo}
    - {outputPath: playbook_dir}
    - {outputPath: static_inventory}
    - {outputPath: access_token}
    - {outputPath: devices}
    - {outputPath: tunnels_created}
    
    command:
    - bash
    - -c
    - |
        set -eo pipefail
        export NAMESPACE=$0
        export PROJECT_ID=$1
        export KUBECONFIG=$2

        export CLUSTER=$(if [ "$PROJECT_ID" == "teknoir" ]; then echo "teknoir-cluster"; else echo "teknoir-dev-cluster"; fi)
        echo "Get credentials to $CLUSTER GKE cluster"
        gcloud container clusters get-credentials $CLUSTER --zone us-central1-c --project $PROJECT_ID
        echo "Kube config generated in $KUBECONFIG"

        echo "Clone git repo: $4"
        git clone $4 $5

        echo "Generate static inventory"
        /etc/ansible/inventory.py --yaml --output /inventory
        mkdir -p $6
        cp -rf /inventory/* $6

        mkdir -p $(dirname $7)
        gcloud auth application-default print-access-token > $7
        
        DEVICES=( $(ansible --list-hosts all --limit $3 | awk 'NR>1') )
        mkdir -p $(dirname $8)
        echo "${DEVICES[@]}" > $8
        
        #echo "Check reverse tunnels"
        #TUNNELS_CREATED=()
        #for DEVICE in $DEVICES ; do
        #  TUNNEL_PORT="$(kubectl get device $DEVICE -o jsonpath='{.spec.keys.data.tunnel}' | base64 -d)"
        #  RE='^[0-9]+$'
        #  if ! [[ $TUNNEL_PORT =~ $RE ]] ; then
        #    echo "Enabling reverse tunnel for: $DEVICE"
        #    TUNNEL_PORT="$(( ( RANDOM % 64511 )  + 1024 ))"
        #    TUNNEL_PORT_B64=$(echo -ne "$TUNNEL_PORT" | base64)
        #    kubectl patch device $DEVICE \
        #      --type merge \
        #      -p "{\"spec\":{\"keys\":{\"data\":{\"tunnel\":\"$TUNNEL_PORT_B64\"}}}}"
        #    TUNNELS_CREATED+=$DEVICE
        #  else
        #    echo "Reverse tunnel already enabled for: $DEVICE"
        #  fi
        #done

        #if (( ${#TUNNELS_CREATED[@]} )); then
        #    echo "Wait 2 minutes for reverse tunnels to warm up"
        #    sleep 2m
        #fi
        
        mkdir -p $(dirname $9)
        echo "${TUNNELS_CREATED[@]}" > $9
""")

run = comp.load_component_from_text("""
name: Run
description: Runs the playbook

inputs:
- {name: namespace, type: String}
- {name: project_id, type: String}
- {name: kube_config, type: Data}
- {name: playbook_path, type: String}
- {name: playbook_dir, type: Data}
- {name: ansible_limit, type: String}
- {name: static_inventory, type: Data}
- {name: access_token, type: Data}

outputs:
- {name: failed_devices, type: Data}
- {name: successful_devices, type: Data}

implementation:
  container:
    image: gcr.io/teknoir/edgebuild-kfp
    args:
    - {inputValue: namespace}
    - {inputValue: project_id}
    - {inputPath: kube_config}
    - {inputPath: playbook_dir}
    - {inputValue: playbook_path}
    - {inputValue: ansible_limit}
    - {inputPath: static_inventory}
    - {inputValue: access_token}
    - {outputPath: failed_devices}
    - {outputPath: successful_devices}
    command:
    - bash
    - -c
    - |
        set -eo pipefail
        export NAMESPACE=$0
        export PROJECT_ID=$1
        export KUBECONFIG=$2
        export ACCESS_TOKEN=$7
        
        echo "Access token: $ACCESS_TOKEN"

        mkdir -p /inventory
        cp -rf $6/* /inventory/

        cd $3
        echo "Run playbook: $4 on $5"
        ansible-playbook -v $4 --inventory /inventory/inventory.yaml --limit $5 || true
        
        mkdir -p $(dirname $8)
        cp failed_devices $8

        mkdir -p $(dirname $9)
        cp successful_devices $9
""")

exit_component = comp.load_component_from_text("""
name: Exit
description: Finally, run this to clean up

inputs:
- {name: status, type: String}
- {name: namespace, type: String}
- {name: project_id, type: String}
- {name: devices, type: Data}
- {name: tunnels_created, type: Data}
- {name: failed_devices, type: Data}
- {name: successful_devices, type: Data}
- {name: add_device_label, type: String}

implementation:
  container:
    image: gcr.io/teknoir/edgebuild-kfp
    args:
    - {inputValue: namespace}
    - {inputValue: project_id}
    - {inputValue: status}
    - {inputPath: devices}
    - {inputPath: tunnels_created}
    - {inputPath: failed_devices}
    - {inputPath: successful_devices}
    - {inputValue: add_device_label}
    command:
    - bash
    - -c
    - |
        set -eo pipefail
        export NAMESPACE=$0
        export PROJECT_ID=$1
        
        echo "Status: $2"
        
        mapfile -t DEVICES < $3
        echo "Devices in play: $DEVICES"

        mapfile -t FAILED_DEVICES < $5
        echo "Failed Devices: $FAILED_DEVICES"

        mapfile -t SUCCESSFUL_DEVICES < $6
        echo "Successful Devices: $SUCCESSFUL_DEVICES"
        
        #mapfile -t TUNNELS_CREATED < $4
        #echo "Closing tunnels for: $TUNNELS_CREATED"
        #for DEVICE in $TUNNELS_CREATED ; do
        #  echo "Disabling reverse tunnel for: $DEVICE"
        #  NA_TUNNEL_PORT_B64=$(echo -ne "NA" | base64)
        #  kubectl patch device $DEVICE \
        #    --type merge \
        #    -p "{\"spec\":{\"keys\":{\"data\":{\"tunnel\":\"$NA_TUNNEL_PORT_B64\"}}}}"
        #done
        
        #if ! [[ -z "$7" ]] ; then
        #    for DEVICE in $SUCCESSFUL_DEVICES ; do
        #      echo "Adding label $7 to $DEVICE"
        #      # TODO: patch labels
        #      # kubectl patch device "{{ ansible_teknoir_device }}" --type merge -p "{\"metadata\":{\"labels\":{\"$7[0]\":\"$7[1]\"}}}"
        #    done
        #fi
""")

## Define the pipeline

In [88]:


project_id = os.getenv('PROJECT_ID', 'teknoir')
namespace = os.getenv('NAMESPACE', 'teknoir-ai')

pipeline_name='Patch device'
pipeline_description='A Kubeflow pipeline running Ansible playbook from a git repo on a subset of devices'
    
@dsl.pipeline(name=pipeline_name, description=pipeline_description)
def ansible_pipeline(playbook_git_repo, playbook_path, ansible_limit, add_device_label=''):
    """A pipeline to run arbitrary playbook from a public git repo."""
    
    prepare_step = prepare(        
        namespace=namespace,
        project_id=project_id,
        ansible_limit=ansible_limit,
        playbook_git_repo=playbook_git_repo
    )
    

    
    #with dsl.ExitHandler(exit_step):
    run_step = run(
        namespace=namespace,
        project_id=project_id,
        kube_config=prepare_step.outputs['kube_config'],
        playbook_path=playbook_path,
        playbook_dir=prepare_step.outputs['playbook_dir'],
        ansible_limit=ansible_limit,
        static_inventory=prepare_step.outputs['static_inventory'],
        access_token=prepare_step.outputs['access_token']
    ).add_pod_annotation('sidecar.istio.io/inject', 'true')

    exit_step = exit_component(
        status="{{workflow.status}}",
        namespace=namespace,
        project_id=project_id,
        devices=prepare_step.outputs['devices'],
        tunnels_created=prepare_step.outputs['tunnels_created'],
        failed_devices=run_step.outputs['failed_devices'],
        successful_devices=run_step.outputs['successful_devices'],
        add_device_label=add_device_label
    ).after(run_step)
        
    prepare_step.execution_options.caching_strategy.max_cache_staleness = "P0D"
    run_step.execution_options.caching_strategy.max_cache_staleness = "P0D"
    exit_step.execution_options.caching_strategy.max_cache_staleness = "P0D"



## Upload pipeline

In [89]:
pipeline_version_file = pipeline_file = 'device_patch.yaml'
# 24h timeout
pipeline_conf = kfp.dsl.PipelineConf().set_timeout(3600*24).set_image_pull_policy(policy="Always")
workflow = kfp.compiler.Compiler().compile(pipeline_func=ansible_pipeline, package_path=pipeline_file, pipeline_conf=pipeline_conf)

client = kfp.Client(namespace='teknoir')
filter = json.dumps({'predicates': [{'key': 'name', 'op': 1, 'string_value': pipeline_name}]})
pipelines = client.pipelines.list_pipelines(filter=filter)

if not pipelines.pipelines:
    pipeline = client.pipeline_uploads.upload_pipeline(pipeline_file, name=pipeline_name, description=pipeline_description)
else:
    pipeline_version_name = pipeline_name + f' - {str(uuid.uuid4())[:6]}'
    pipeline_version = client.pipeline_uploads.upload_pipeline_version(pipeline_version_file,
                                                                       name=pipeline_version_name,
                                                                       pipelineid=pipelines.pipelines[0].id)

ERROR:root:Failed to read a token from file '/var/run/secrets/kubeflow/pipelines/token' ([Errno 2] No such file or directory: '/var/run/secrets/kubeflow/pipelines/token').
